# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = list(df[['song_id', 'artist_id', 'year', 'duration']].values[0])
song_data

['SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
song_table_insert.format(*song_data)

"insert into songs (song_id, artist_id, year, duration) values ('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1', 0, 218.93179)\n"

In [9]:
cur.execute(song_table_insert.format(*song_data))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]

array(['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan], dtype=object)

In [11]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
artist_table_insert.format(*artist_data)

"insert into artists (artist_id, name, location, latitude, longitude) values ('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', 'nan', 'nan')\n"

In [13]:
cur.execute(artist_table_insert.format(*artist_data))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
sorted(log_files)

['/home/workspace/data/log_data/2018/11/.ipynb_checkpoints/2018-11-01-events-checkpoint.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-02-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-05-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-07-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-08-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-12-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.js

In [16]:
filepath = log_files[0]

In [17]:
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [18]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [20]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [21]:
time_data = [(ts.hour, ts.day, ts.week, ts.month, ts.year, ts.weekday()) for ts in list(t)]

In [22]:
column_labels = (['hour', 'day', 'week', 'month', 'year', 'weekday'])

In [23]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

,hour,day,week,month,year,weekday
0,0,30,48,11,2018,4
1,1,30,48,11,2018,4
2,1,30,48,11,2018,4
3,1,30,48,11,2018,4
4,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert.format(*list(row)))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [26]:
user_df = user_df.drop_duplicates()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert.format(*list(row)))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():
    print(row.song.replace("'", "''"))

Jim Henson''s Dead
Shell Shock
Women Lose Weight (Feat: Slick Rick)
Won''t Go Home Without You
Hey_ Soul Sister
I''m In Miami Bitch
Sexy Bitch
The Cure & The Cause (Dennis Ferrer Remix)
Staring At Me
The Second You Sleep
Wide Eyes
La Resistance (Medley) (LP Version)
Restless
Noc...
Like You
God Put A Smile Upon Your Face
Playboy
Every Time You Go
Natural Anthem (Album)
Taylor
Naked As We Can
Fantasy
Epilogue
Groovin''
Ain''t No Way (You Won''t Love Me)
Rome
Highway Through My Home
143
Dreams-Come-True-Girl
I Don''t Feel Like Dancin''
Right On Time (Album Version)
Black Lines
Uprising
Man Makes The Zoo
Blacks/Radio
Love Dealer (Featuring Justin Timberlake)
ReprÃÂ©sente
Hey_ Soul Sister
Up And Gone
Te Conoci
Bottle Pop
My Love Will Follow You
Found Home
My December
The Perfect Kiss
You''re Gonna Make Me Lonesome When You Go
Shortie Like Mine
Domination
Inside My Love (Digitally Remastered 93)
Structure
Killing In The Name
Girl You Know It''s True
Waterfall (Spirit Of The Rainforest Albu

In [29]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select.format(row.song.replace("'", "''"), row.artist.replace("'", "''"), row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent'])

In [30]:
songplay_data

(1543607664796,
 '5',
 'free',
 None,
 None,
 985,
 'Detroit-Warren-Dearborn, MI',
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"')

In [31]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select.format(row.song.replace("'", "''"), row.artist.replace("'", "''"), row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent'])
    cur.execute(songplay_table_insert.format(*songplay_data))
    conn.commit()

In [32]:
songplay_table_insert.format(*songplay_data)

'insert into songplays (start_time, user_id, level, song_id, artist_id, songplay_id, location, user_agent) values (1543607664796, 5, \'free\', \'None\', \'None\', \'985\', \'Detroit-Warren-Dearborn, MI\', \'"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"\')\n'

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [33]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.